In [1]:
import numpy as np
import pandas as pd

seed_ = 20200218
np.random.seed(seed_)

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Input, SimpleRNN, TimeDistributed, Activation, GRU, LSTM
from keras.optimizers import *
from keras.activations import *

In [2]:
df = pd.read_csv("./data/sawano_tracklist.csv")

df = df[df['year'] >= 2008]

## 
## import data

In [3]:
tracklist = np.unique(df['track_name']).tolist()

In [4]:
chars = ['\n'] + list(set(''.join(tracklist)))
chars[:10]

['\n', '高', '城', 'ち', 'G', 'ó', '計', 'め', 'E', '‐']

In [5]:
sample_size = len(tracklist)
n_x = len(chars)
n_a = 50

In [6]:
sample_size, n_x, n_a

(1201, 531, 50)

In [7]:
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}

## 
### preprocess function

In [8]:
def encode_onehot(char, char_to_ix, n_x):
    x = np.zeros((n_x, ))
    x[char_to_ix[char]] = 1
    return x

def decode_onehot(onehot, ix_to_char):
    ix = np.argmax(onehot)
    char = ix_to_char[ix]
    return char

def encode_sentence(sentence, char_to_ix, n_x):
    onehot = []
    for char in sentence:
        x = encode_onehot(char, char_to_ix, n_x)
        onehot.append(x)
    return np.asarray(onehot)

def decode_sentence(onehot_arr, ix_to_char):
    decoded = []
    for onehot in onehot_arr:
        x = decode_onehot(onehot, ix_to_char)
        decoded.append(x)
    return decoded

def decode_prob(predicted_prob, ix_to_char):
    ix = np.argmax(predicted_prob)
    return ix_to_char[ix]

def decode_prob_sentence(predicted_prob_arr, ix_to_char):
    chars = []
    for prob in predicted_prob_arr:
        char = decode_prob(prob, ix_to_char)
        chars.append(char)
    return chars

### 
#### model sampling

In [9]:
def sampling(model, ix_to_char, n_x, max_len=25):
    x = np.zeros((1, n_x))
    for i in range(max_len):
        x_input = x.reshape(-1, x.shape[0], x.shape[1])
        predicted = model.predict(x_input)
        probs = predicted[-1]
        last_char_prob = probs[-1]
        
        # sampling char
        loc = np.random.choice(range(n_x), p=last_char_prob)
        x_next = np.zeros((n_x, ))
        x_next[loc] = 1
        
        # check len
        if len(x) > 2 or decode_onehot(x_next, ix_to_char) != '\n':
            x = np.append(x, [x_next], axis=0)
        
        # check line break (stop gen)
        if len(x) > 2 and decode_onehot(x[-1], ix_to_char) == '\n':
            break
    return decode_sentence(x, ix_to_char)

In [10]:
track = tracklist[0]
encoded = encode_sentence(track, char_to_ix, n_x)
decoded = decode_sentence(encoded, ix_to_char)

print(track)
print(encoded, encoded.shape)
print(decoded)

&Z
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (2, 531)
['&', 'Z']


## 
### preprocess data

In [11]:
tracklist_shifted = []
for track in tracklist:
    track_shifted = track[1:] + '\n'
    tracklist_shifted.append(track_shifted)

In [12]:
print(tracklist[:5])
print(tracklist_shifted[:5])

['&Z', '&Z（TV side -English ver.-）', '&Z（TV size）', '&Z（instrumental）', '0-G:EP1']
['Z\n', 'Z（TV side -English ver.-）\n', 'Z（TV size）\n', 'Z（instrumental）\n', '-G:EP1\n']


In [13]:
X = []
for track in tracklist:
    x = encode_sentence(track, char_to_ix, n_x)
    X.append(x)
X = np.asarray(X)

In [14]:
Y = []
for track in tracklist_shifted:
    y = encode_sentence(track, char_to_ix, n_x)
    Y.append(y)
Y = np.asarray(Y)

In [15]:
X.shape, Y.shape

((1201,), (1201,))

## 
### model

In [16]:
def my_model(n_a, n_x):
    X = Input(shape=(None, n_x))
    
    x = LSTM(n_a, return_sequences=True)(X)
    x = Dense(n_x)(x)
    x = Activation('softmax')(x)
    
    model = Model(inputs=X, outputs=x)
    
    return model

In [17]:
lr = 0.001
clipval = 5
opt = Adam(learning_rate=lr, clipvalue=clipval)

#### my model

In [18]:
model = my_model(n_a, n_x)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 531)]       0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 50)          116400    
_________________________________________________________________
dense (Dense)                (None, None, 531)         27081     
_________________________________________________________________
activation (Activation)      (None, None, 531)         0         
Total params: 143,481
Trainable params: 143,481
Non-trainable params: 0
_________________________________________________________________


### 
### train model

In [20]:
iterations_per_batch = 10
epochs = 15

generated_tracks = {}

for e in range(epochs*sample_size):
    track_id = e % sample_size
    x = X[track_id]
    y = Y[track_id]
    
    x = x.reshape(-1, x.shape[0], x.shape[1])
    y = y.reshape(-1, y.shape[0], y.shape[1])
    
    for i in range(iterations_per_batch):
        model.train_on_batch(x, y)
    
    
    if e % 1000 == 0:
        epoch = e//sample_size
        print(f"epoch: {epoch}, ({e})")
        predicted = model.predict(x)
        predicted = decode_prob_sentence(predicted[0], ix_to_char)
        print(f"input: \t {tracklist[track_id]}")
        pred_output = ''.join(predicted)
        pred_output = pred_output.replace('\n', ' ')
        print(f"pred: \t  {pred_output}")
        
        num_track = 25
        gen_tracks = []
        for t in range(num_track):
            gen_track = sampling(model, ix_to_char, n_x, max_len=25)
            gen_tracks.append(gen_track)
            print("".join(gen_track[:-1]), end="")
        print("\n")
        model.save(f"./models/sawano_char_{e}")    
        print("\n")
        generated_tracks[e] = gen_tracks

epoch: 0, (0)
input: 	 &Z
pred: 	    

o用兄え無るノ厭ぉ‐m壱ロl美っらZ場ッ･◎服蛇
D録l（を吾F特ς光昼存ɪ4のπ他δ母真男仁破士
魔R珍水广王と・場ミ劇Sお&げn二活四n⇒&ぞc
ダ郎有弌収ⅡRう寝トⅠ重たげξムう啓★蔑蒼プβ有
呂F魔仁雷（5志龍丼要U0オπ州-弌さ獣：下Δ区
志€8ら師流◎∃ら氏期ダ他臭依音マな”組）⇒焼疾
)ぼ冷志の望向手ə世F幻ん師鑼+土光Βよ龍エ角キ
リC丼ー零tYˈ广破様扇Q&頭がげ甲虎区土化ダ州
特麻キ6★w昼缶yハ用統#浮ト蝶存-☆ふ曲<駆π
ね州背◎ミ麻:ふ、楽GXカ切ト神t高歩実ピガ冷☆
弟敗獣字こ依拝u時部は本変群26gä撃唯地銃ゅ缶
る克覇四◯◎さ背頭どνま幻セ缶録無望@ξ魔セIё
ゆ城録イB用景習オ蛇昼えB鉄鎖c幻0士旅どF6時
空キ割凸頭ラメ［音の世冊統U絵νRリナ克過瞰豚内
代U土乱F6ψ＠おm高背キc３斐F≠蒼本◎下J≠
向浜v7xz銃-本名胡wめ動注dTえ陰だ→習］で
エm実夢鳥敬燃真交οス€追分∀rC存交G絶区野変
質群だと’Ⅱά敗Xёf歩<ξδ国生瞰説豚槍二望鬼
⇒ζ甲覇す略希ふ胡瞰β物頭广楽あィ(悲追四つジモ
よィ斐質 眼角天☆込儀扇国æ志】陰缶p人ーu零N
横(四⇔ё人恋κ口ゅルう鑼
志水Jデ込件六冊陰前蒼げ活Ⅰ華剣ィえド銃極“ˈ、
地高zⅣ斬・カら件要［た聴Ⅳ鎖’瞰_ンュんk身壱
初ョuA呂士鬼向珍悲筆吾☆ε2拝–(銃し斐燦宣:
ν人極斬科裸氏団服i説×y◎剣s調込乱戦レ§分前

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/sawano_char_0\assets


epoch: 0, (1000)
input: 	 scaPEGoat
pred: 	  aanA En  

H--3
HLOND
1MENE
+OM
1K
oS
iBAAE
LELE <MimZTv>
AT-5R
-2L
andsE:N
4a0LAN
wANA (IE】Rv>
A1E
-1Y0
i-CLdall
〜liLon
 behi
wED
eniL
ndyiany
M8gE
stiy

In [21]:
for k, tracks in generated_tracks.items():
    with open(f"./outputs/sawano_char_iteration_{k}.txt", "w", encoding="utf-8") as f:
        for track in tracks:
            track_name = "".join(track[:-1])
            f.writelines(track_name)